# 🚀 PPO Ultimate Training - Gold Trading AI

**Notebook de Entrenamiento PPO con todas las mejoras del checklist**

Este notebook implementa TODAS las recomendaciones de:
`PPO_GOLD_TRADING_IMPROVEMENTS_CHECKLIST.md`

---

## ✅ Mejoras Implementadas:

### Hiperparámetros PPO (Sección 1):
- ✅ **1.1 Gamma = 0.6** (optimizado para intraday-swing)
- ✅ **1.2 Learning Rate Annealing** (3e-4 → 1e-5)
- ✅ **1.3 Target KL = 0.01** (early stopping)
- ✅ **1.4 Entropy Coef = 0.02** (exploración balanceada)
- ✅ **1.5 Batch=128, N_Steps=4096** (gradientes estables)

### Reward Function (Sección 2):
- ✅ **5 componentes**: Sortino, DD penalty, TC penalty, Holding bonus, Breach penalty
- ✅ **Differential Sharpe Ratio (DSR)**

### Features (152 total):
- ✅ Multi-timeframe (M5, M15, H1, H4, D1, W1)
- ✅ Indicadores optimizados para oro (RSI 21, MACD 16/34/13, BB 13)
- ✅ Macro correlations (DXY, VIX, TIPS, Oil, etc.)
- ✅ Economic calendar features

---

## ⚙️ Setup:
1. **GPU**: Runtime → Change runtime type → **GPU (A100 recomendado)**
2. **Ejecutar celdas en orden**
3. **Tiempo estimado**: 3-5 horas en A100

## 1️⃣ Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Navegar al Proyecto

**⚠️ IMPORTANTE**: Cambia `PROJECT_PATH` a la ruta de tu proyecto en Drive

In [ ]:
import os

# ⚠️ CAMBIA ESTA RUTA a donde está tu proyecto
PROJECT_PATH = '/content/drive/MyDrive/XAUX'

# Verificar que existe
if not os.path.exists(PROJECT_PATH):
    print(f"❌ ERROR: No existe la ruta: {PROJECT_PATH}")
    print("\n📁 Carpetas disponibles en tu Drive:")
    !ls /content/drive/MyDrive/
else:
    os.chdir(PROJECT_PATH)
    print(f"✅ Directorio: {os.getcwd()}")
    print(f"\n📁 Archivos:")
    !ls -la

## 3️⃣ Instalar Dependencias

In [ ]:
# PyTorch con CUDA
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Stable Baselines 3 (PPO)
!pip install stable-baselines3[extra]

# Otras dependencias
!pip install pandas numpy tqdm matplotlib seaborn gymnasium
!pip install yfinance  # Para datos macro

print("\n✅ Dependencias instaladas!")

## 4️⃣ Verificar GPU

In [ ]:
import torch

print(f"🖥️  PyTorch: {torch.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {gpu_name}")
    print(f"💾 Memoria: {gpu_memory:.2f} GB")
    device = 'cuda'
else:
    print("⚠️  GPU no disponible, usando CPU (será lento)")
    device = 'cpu'

print(f"\n✅ Device: {device}")

## 5️⃣ Verificar Stable-Baselines3

In [ ]:
import stable_baselines3
from stable_baselines3 import PPO

print(f"✅ Stable-Baselines3 version: {stable_baselines3.__version__}")
print(f"✅ PPO disponible: {PPO is not None}")

## 6️⃣ Test de Features (Opcional)

Verifica que el sistema de features funciona correctamente.

In [ ]:
print("🧪 Testeando sistema de features...")
print("   (Esto toma ~2-3 minutos)\n")

from features.ultimate_150_features import make_ultimate_features

X, returns, timestamps = make_ultimate_features(base_timeframe='M5')

print(f"\n✅ Features cargados exitosamente!")
print(f"📊 Shape: {X.shape}")
print(f"📊 Total features: {X.shape[1]}")
print(f"📊 Total samples: {X.shape[0]:,}")
print(f"📅 Rango: {timestamps[0]} a {timestamps[-1]}")

## 7️⃣ Configuración de Entrenamiento

### Configuraciones recomendadas:

| Config | Timesteps | Window | Tiempo A100 | Uso |
|--------|-----------|--------|-------------|-----|
| Test rápido | 100,000 | 64 | ~20 min | Verificar que funciona |
| Corto | 500,000 | 64 | ~1.5 horas | Ver aprendizaje inicial |
| Medio | 1,000,000 | 128 | ~3 horas | Resultados decentes |
| **Recomendado** | **3,000,000** | **128** | **~9 horas** | **Producción** |
| Extenso | 5,000,000 | 128 | ~15 horas | Máximo rendimiento |

### Sobre el Window:
- **64 barras** = 5.3 horas de contexto (mínimo)
- **128 barras** = 10.6 horas de contexto (recomendado, cubre sesión completa)
- **256 barras** = 21 horas de contexto (si quieres más contexto)

In [ ]:
# ========================================
# CONFIGURACIÓN DE ENTRENAMIENTO
# ========================================

# Timesteps totales
TIMESTEPS = 3_000_000  # 3M recomendado para window=128

# Ventana de observación (contexto que ve el agente)
WINDOW = 128  # 128 barras = 10.6 horas de contexto M5

# Device
DEVICE = device  # Detectado automáticamente arriba

# ========================================
# PPO HYPERPARAMETERS (Del Checklist)
# ========================================

PPO_CONFIG = {
    # Sección 1.1: Gamma optimizado para intraday-swing
    'gamma': 0.6,
    
    # Sección 1.3: Target KL para early stopping
    'target_kl': 0.01,
    
    # Sección 1.4: Entropy coefficient
    'ent_coef': 0.02,
    
    # Sección 1.5: Batch size y n_steps
    'n_steps': 4096,
    'batch_size': 128,
    
    # Parámetros estándar
    'n_epochs': 10,
    'gae_lambda': 0.95,
    'clip_range': 0.2,
    'vf_coef': 0.5,
    'max_grad_norm': 0.5,
}

print("⚙️  Configuración PPO (del Checklist):")
print("="*50)
for key, value in PPO_CONFIG.items():
    print(f"   {key}: {value}")
print("="*50)
print(f"\n📊 Timesteps: {TIMESTEPS:,}")
print(f"🪟 Window: {WINDOW} barras ({WINDOW * 5 / 60:.1f} horas de contexto)")
print(f"🖥️  Device: {DEVICE}")

# Estimación de tiempo
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
if 'A100' in gpu_name:
    hours = TIMESTEPS / 1_000_000 * 3
elif 'H100' in gpu_name:
    hours = TIMESTEPS / 1_000_000 * 2
else:
    hours = TIMESTEPS / 1_000_000 * 5

print(f"\n⏱️  Tiempo estimado: ~{hours:.1f} horas")

## 8️⃣ Entrenar PPO

**⚠️ IMPORTANTE**: Esta celda tarda varias horas. No cierres la pestaña.

Los checkpoints se guardan cada 10,000 timesteps en tu Drive.

In [ ]:
!python train/train_ppo_ultimate.py \
    --timesteps {TIMESTEPS} \
    --window {WINDOW} \
    --device {DEVICE}

## 9️⃣ Resumir Entrenamiento (si se interrumpe)

Si el entrenamiento se interrumpe, puedes continuar desde el último checkpoint:

In [ ]:
# Ver checkpoints disponibles
!ls -lh train/ppo_ultimate/*.zip 2>/dev/null || echo "No hay checkpoints aún"

# Descomentar y ajustar para resumir:
# CHECKPOINT = "train/ppo_ultimate/ppo_ultimate_100000_steps.zip"
# !python train/train_ppo_ultimate.py --timesteps {TIMESTEPS} --device {DEVICE} --resume {CHECKPOINT}

## 🔟 Ver Resultados

In [ ]:
import json

# Cargar métricas
metrics_path = 'train/ppo_ultimate/ppo_ultimate_metrics.json'

try:
    with open(metrics_path, 'r') as f:
        metrics = json.load(f)
    
    print("="*70)
    print("📊 RESULTADOS PPO ULTIMATE")
    print("="*70)
    
    print(f"\n🎯 VALIDATION (2022):")
    val = metrics['validation']
    print(f"   Total Return: {val['total_return']:.2f}%")
    print(f"   Sharpe Ratio: {val['sharpe_ratio']:.2f}")
    print(f"   Sortino Ratio: {val['sortino_ratio']:.2f}")
    print(f"   Max Drawdown: {val['max_drawdown']:.2f}%")
    print(f"   Calmar Ratio: {val['calmar_ratio']:.2f}")
    print(f"   Profit Factor: {val['profit_factor']:.2f}")
    print(f"   Win Rate: {val['win_rate']:.1f}%")
    print(f"   Trades: {val['trades']}")
    
    print(f"\n🎯 TEST (2023-2025 Out-of-Sample):")
    test = metrics['test']
    print(f"   Total Return: {test['total_return']:.2f}%")
    print(f"   Sharpe Ratio: {test['sharpe_ratio']:.2f}")
    print(f"   Sortino Ratio: {test['sortino_ratio']:.2f}")
    print(f"   Max Drawdown: {test['max_drawdown']:.2f}%")
    print(f"   Calmar Ratio: {test['calmar_ratio']:.2f}")
    print(f"   Profit Factor: {test['profit_factor']:.2f}")
    print(f"   Win Rate: {test['win_rate']:.1f}%")
    print(f"   Trades: {test['trades']}")
    
    print(f"\n⚙️  Configuración usada:")
    config = metrics.get('config', {})
    for k, v in config.items():
        print(f"   {k}: {v}")
        
except FileNotFoundError:
    print("❌ Archivo de métricas no encontrado.")
    print("   Ejecuta el entrenamiento primero.")

## 1️⃣1️⃣ Comparar PPO vs DreamerV3

Si entrenaste ambos modelos, compara sus resultados:

In [ ]:
import json
import pandas as pd

results = {}

# Cargar PPO
try:
    with open('train/ppo_ultimate/ppo_ultimate_metrics.json', 'r') as f:
        ppo = json.load(f)
        results['PPO'] = ppo['test']
except:
    print("PPO metrics not found")

# Cargar DreamerV3
try:
    with open('train/dreamer_ultimate/ultimate_150_xauusd_metrics.json', 'r') as f:
        dreamer = json.load(f)
        results['DreamerV3'] = dreamer['test']
except:
    print("DreamerV3 metrics not found")

if len(results) >= 2:
    df = pd.DataFrame(results).T
    
    print("="*70)
    print("📊 COMPARACIÓN PPO vs DreamerV3 (Test Set)")
    print("="*70)
    
    display_cols = ['total_return', 'sharpe_ratio', 'sortino_ratio', 
                    'max_drawdown', 'profit_factor', 'win_rate', 'trades']
    
    print(df[display_cols].round(2).to_string())
    
    # Ganador
    ppo_sharpe = results['PPO']['sharpe_ratio']
    dreamer_sharpe = results['DreamerV3']['sharpe_ratio']
    
    print(f"\n🏆 Mejor Sharpe: {'PPO' if ppo_sharpe > dreamer_sharpe else 'DreamerV3'}")
else:
    print("\n⚠️  Entrena ambos modelos para comparar")

## 1️⃣2️⃣ Descargar Modelo

In [ ]:
from google.colab import files
import glob

# Buscar modelo final
model_files = glob.glob('train/ppo_ultimate/*.zip')

if model_files:
    latest = sorted(model_files)[-1]
    print(f"📥 Descargando: {latest}")
    files.download(latest)
else:
    print("❌ No se encontraron modelos")

## 1️⃣3️⃣ TensorBoard (Opcional)

Visualiza las curvas de entrenamiento:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir train/ppo_tensorboard/

---

## 📝 Notas Importantes

### Diferencias PPO vs DreamerV3:

| Aspecto | PPO | DreamerV3 |
|---------|-----|----------|
| Tipo | Model-Free | Model-Based |
| Velocidad | Más rápido por step | Más lento por step |
| Eficiencia de datos | Menor | Mayor |
| Complejidad | Menor | Mayor |
| Estabilidad | Alta | Alta |

### Cuándo usar cada uno:
- **PPO**: Cuando quieres resultados rápidos y estables
- **DreamerV3**: Cuando quieres máxima eficiencia de datos y mejor generalización

### Siguiente paso:
Entrena ambos modelos y compara resultados en el Test Set (2023-2025)

---

🎉 **¡Felicidades! Tu modelo PPO está listo.**